# 🫁 MobileNet V2 - Chest X-ray Multi-Label Classification

**Dataset:** CheXpert 3-class (Cardiomegaly, Edema, Pneumothorax) - No Finding Excluded

**Model:** MobileNet V2 (pretrained on ImageNet)

**Outputs:**
- 3 Binary Classification Reports (one per class)
- 3 Binary Confusion Matrices (one per class)
- Training/Validation Loss & Accuracy Curves
- ROC Curves for all 3 classes

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

# Torchvision
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

# Sklearn metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_curve, auc, accuracy_score, f1_score,
    precision_score, recall_score
)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.0+cu126
CUDA available: True


## ⚙️ Configuration

In [9]:
class Config:
    # Paths - UPDATE THIS TO YOUR DATASET NAME
    DATA_DIR = "/kaggle/input/datasets/shiv28/chest-xray-4class-100k/"  # Your uploaded dataset
    OUTPUT_DIR = "/kaggle/working"
    
    # Model
    MODEL_NAME = "mobilenet"
    NUM_CLASSES = 3
    LABELS = ["Cardiomegaly", "Edema", "Pneumothorax"]
    
    # Training
    BATCH_SIZE = 32
    NUM_EPOCHS = 25
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-5
    
    # Image
    IMG_SIZE = 224
    
    # Device
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Random seed
    SEED = 42

config = Config()

# Set seeds for reproducibility
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(config.SEED)

print(f"🖥️ Device: {config.DEVICE}")
print(f"🤖 Model: {config.MODEL_NAME}")
print(f"🏷️ Labels: {config.LABELS}")

🖥️ Device: cuda
🤖 Model: mobilenet
🏷️ Labels: ['Cardiomegaly', 'Edema', 'Pneumothorax']


## 📦 Dataset Class

In [10]:
class ChestXrayDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.labels = config.LABELS
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Get image path
        img_path = row['new_path']
        full_path = os.path.join(self.img_dir, img_path)
        
        # Load image
        try:
            image = Image.open(full_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {full_path}: {e}")
            image = Image.new('RGB', (config.IMG_SIZE, config.IMG_SIZE), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        # Get labels (multi-hot encoding)
        labels = []
        for label in self.labels:
            val = row[label] if label in row.index else 0
            labels.append(1.0 if val == 1.0 else 0.0)
        
        labels = torch.tensor(labels, dtype=torch.float32)
        
        return image, labels

## 🔄 Data Transforms

In [11]:
train_transform = transforms.Compose([
    transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
   
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

## 📊 Load Data

In [12]:
print("=" * 60)
print("LOADING DATA")
print("=" * 60)

# Check what's in the data directory
print(f"\nContents of {config.DATA_DIR}:")
if os.path.exists(config.DATA_DIR):
    for item in os.listdir(config.DATA_DIR):
        print(f"  {item}")
else:
    print(f"  ⚠️ Directory {config.DATA_DIR} not found. Please check dataset path.")

# Load metadata
metadata_path = os.path.join(config.DATA_DIR, "metadata.csv")
if os.path.exists(metadata_path):
    df = pd.read_csv(metadata_path)
    print(f"\n📈 Total samples: {len(df)}")

    # Check label distribution
    print("\n📊 Label distribution:")
    for label in config.LABELS:
        if label in df.columns:
            count = (df[label] == 1.0).sum()
            print(f"  {label}: {count} ({count/len(df)*100:.2f}%)")
else:
    # Fallback for testing without data
    print("⚠️ Metadata file not found. Creating dummy data structure for compilation check.")
    df = pd.DataFrame(columns=['new_path'] + config.LABELS)

LOADING DATA

Contents of /kaggle/input/datasets/shiv28/chest-xray-4class-100k/:
  images
  metadata.csv

📈 Total samples: 95060

📊 Label distribution:
  Cardiomegaly: 23451 (24.67%)
  Edema: 49717 (52.30%)
  Pneumothorax: 17700 (18.62%)


In [13]:
# Split data
if len(df) > 0:
    train_df, val_df = train_test_split(
        df, test_size=0.2, random_state=config.SEED, 
        stratify=None
    )
else:
    train_df, val_df = df, df

print(f"🏋️ Train samples: {len(train_df)}")
print(f"🧪 Validation samples: {len(val_df)}")

# Create datasets
train_dataset = ChestXrayDataset(train_df, config.DATA_DIR, train_transform)
val_dataset = ChestXrayDataset(val_df, config.DATA_DIR, val_transform)

# Create dataloaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.BATCH_SIZE, 
    shuffle=True, 
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=config.BATCH_SIZE, 
    shuffle=False, 
    num_workers=2,
    pin_memory=True
)

print(f"\n✅ DataLoaders created!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")

🏋️ Train samples: 76048
🧪 Validation samples: 19012

✅ DataLoaders created!
   Train batches: 2377
   Val batches: 595


## 🤖 Model Definition - MobileNet V2

In [14]:
# ── Imports (add to your existing imports) ──────────────────────────────────
from torchvision.models import swin_b, Swin_B_Weights

# ── Update Config ────────────────────────────────────────────────────────────
class Config:
    DATA_DIR = "/kaggle/input"
    OUTPUT_DIR = "/kaggle/working"
    
    MODEL_NAME = "swin_transformer_base"
    NUM_CLASSES = 3
    LABELS = ["Cardiomegaly", "Edema", "Pneumothorax"]
    
    BATCH_SIZE = 16          # Swin-B is heavier, reduce if OOM
    NUM_EPOCHS = 25
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-5
    
    IMG_SIZE = 224           # Swin-B uses 224x224
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    SEED = 42

config = Config()

# ── Model ─────────────────────────────────────────────────────────────────────
class SwinTransformerBaseChestXray(nn.Module):
    def __init__(self, num_classes=3, pretrained=True):
        super().__init__()
        
        # Load pretrained Swin-B
        weights = Swin_B_Weights.IMAGENET1K_V1 if pretrained else None
        self.backbone = swin_b(weights=weights)
        
        # Replace the head
        in_features = self.backbone.head.in_features  # 1024 for Swin-B
        self.backbone.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)

# ── Instantiate ───────────────────────────────────────────────────────────────
model = SwinTransformerBaseChestXray(num_classes=config.NUM_CLASSES, pretrained=True)
model = model.to(config.DEVICE)

# Verify
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ Model: Swin Transformer Base")
print(f"   Total params:     {total_params:,}")
print(f"   Trainable params: {trainable_params:,}")

✅ Model: Swin Transformer Base
   Total params:     87,269,563
   Trainable params: 87,269,563


## 🎯 Training Setup

In [15]:
# Loss function (BCE for multi-label)
criterion = nn.BCEWithLogitsLoss()

# Optimizer
optimizer = optim.AdamW(
    model.parameters(), 
    lr=config.LEARNING_RATE, 
    weight_decay=config.WEIGHT_DECAY
)

# Scheduler
scheduler = CosineAnnealingLR(optimizer, T_max=config.NUM_EPOCHS, eta_min=1e-6)

print("✅ Training setup complete!")
print(f"   Loss: BCEWithLogitsLoss")
print(f"   Optimizer: AdamW (LR={config.LEARNING_RATE})")
print(f"   Scheduler: CosineAnnealingLR")

✅ Training setup complete!
   Loss: BCEWithLogitsLoss
   Optimizer: AdamW (LR=0.0001)
   Scheduler: CosineAnnealingLR


## 🔧 Training Functions

In [16]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    # Use tqdm for progress bar if loader is not empty
    if len(loader) > 0:
        pbar = tqdm(loader, desc="Training")
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Collect predictions
            preds = torch.sigmoid(outputs).detach().cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        epoch_loss = running_loss / len(loader)
    else:
        epoch_loss = 0.0
    
    # Calculate accuracy (threshold = 0.5)
    if len(all_preds) > 0:
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        binary_preds = (all_preds > 0.5).astype(int)
        accuracy = (binary_preds == all_labels).mean()
    else:
        accuracy = 0.0
    
    return epoch_loss, accuracy

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        if len(loader) > 0:
            pbar = tqdm(loader, desc="Validation")
            for images, labels in pbar:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item()
                
                probs = torch.sigmoid(outputs).cpu().numpy()
                all_probs.extend(probs)
                all_preds.extend((probs > 0.5).astype(int))
                all_labels.extend(labels.cpu().numpy())
            
            epoch_loss = running_loss / len(loader)
        else:
            epoch_loss = 0.0
    
    # Calculate accuracy
    if len(all_preds) > 0:
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)
        accuracy = (all_preds == all_labels).mean()
    else:
        accuracy = 0.0
        all_preds, all_labels, all_probs = np.array([]), np.array([]), np.array([])
    
    return epoch_loss, accuracy, all_preds, all_probs, all_labels

## 🏋️ Training Loop

In [ ]:
print("=" * 60)
print("🚀 TRAINING STARTED - MobileNet V2")
print("=" * 60)

history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': []
}

best_val_loss = float('inf')
best_model_state = None

# Check if we have data to train
if len(train_loader) > 0:
    for epoch in range(config.NUM_EPOCHS):
        print(f"\n📅 Epoch {epoch+1}/{config.NUM_EPOCHS}")
        print("-" * 40)
        
        # Train
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, config.DEVICE
        )
        
        # Validate
        val_loss, val_acc, val_preds, val_probs, val_labels = validate_epoch(
            model, val_loader, criterion, config.DEVICE
        )
        
        # Update scheduler
        scheduler.step()
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Print metrics
        print(f"📉 Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"📊 Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        print(f"📈 LR: {scheduler.get_last_lr()[0]:.6f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            print("✅ Best model saved!")
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        
        # Save model
        model_path = os.path.join(config.OUTPUT_DIR, "mobilenet_chestxray.pth")
        torch.save(best_model_state, model_path)
        print(f"\n💾 Model saved to: {model_path}")
else:
    print("⚠️ No training data available. Skipping training loop.")

🚀 TRAINING STARTED - MobileNet V2

📅 Epoch 1/25
----------------------------------------


Validation: 100%|██████████| 595/595 [02:06<00:00,  4.71it/s]


📉 Train Loss: 0.4314 | Train Acc: 0.8041
📊 Val Loss: 0.3941 | Val Acc: 0.8281
📈 LR: 0.000100
✅ Best model saved!

📅 Epoch 2/25
----------------------------------------


Training:  88%|████████▊ | 2081/2377 [21:49<03:06,  1.59it/s, loss=0.4040]

## 📈 Plot Training Curves

In [ ]:
if len(history['train_loss']) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Loss curve
    axes[0].plot(history['train_loss'], label='Train Loss', marker='o', linewidth=2, color='#e74c3c')
    axes[0].plot(history['val_loss'], label='Val Loss', marker='s', linewidth=2, color='#3498db')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('📉 Training & Validation Loss', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)

    # Accuracy curve
    axes[1].plot(history['train_acc'], label='Train Accuracy', marker='o', linewidth=2, color='#2ecc71')
    axes[1].plot(history['val_acc'], label='Val Accuracy', marker='s', linewidth=2, color='#9b59b6')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].set_title('📈 Training & Validation Accuracy', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(config.OUTPUT_DIR, 'mobilenet_training_curves.png'), dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Saved: mobilenet_training_curves.png")
else:
    print("⚠️ No training history to plot.")

## 🔍 Final Evaluation

In [ ]:
print("=" * 60)
print("🔍 FINAL EVALUATION")
print("=" * 60)

# Get final predictions
if len(val_loader) > 0 and best_model_state is not None:
    val_loss, val_acc, all_preds, all_probs, all_labels = validate_epoch(
        model, val_loader, criterion, config.DEVICE
    )

    print(f"\n📊 Final Validation Loss: {val_loss:.4f}")
    print(f"📊 Final Validation Accuracy: {val_acc:.4f}")
else:
    print("⚠️ Skipping evaluation due to missing data or model.")
    all_preds, all_probs, all_labels = [], [], []

## 📊 4 Binary Confusion Matrices

In [ ]:
if len(all_preds) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    # axes is already 1D for 1x3 subplot

    colors = ['Blues', 'Greens', 'Oranges', 'Purples']

    for i, label in enumerate(config.LABELS):
        # Get binary predictions and labels for this class
        class_preds = all_preds[:, i]
        class_labels = all_labels[:, i]
        
        # Compute confusion matrix
        cm = confusion_matrix(class_labels, class_preds)
        
        # Plot
        sns.heatmap(
            cm, annot=True, fmt='d', cmap=colors[i],
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'],
            ax=axes[i], annot_kws={'size': 16}
        )
        axes[i].set_xlabel('Predicted', fontsize=12)
        axes[i].set_ylabel('Actual', fontsize=12)
        axes[i].set_title(f'🏷️ {label}\nConfusion Matrix', fontsize=14, fontweight='bold')

    plt.suptitle(' - Confusion Matrices', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(os.path.join(config.OUTPUT_DIR, 'confusion_matrices.png'), dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Saved: econfusion_matrices.png")

## 📋 4 Binary Classification Reports

In [ ]:
if len(all_preds) > 0:
    print("=" * 60)
    print("📋 CLASSIFICATION REPORTS (Binary)")
    print("=" * 60)

    reports = []

    for i, label in enumerate(config.LABELS):
        print(f"\n{'='*50}")
        print(f"🏷️ CLASSIFICATION REPORT: {label}")
        print('='*50)
        
        class_preds = all_preds[:, i]
        class_labels = all_labels[:, i]
        
        # Print detailed report
        report = classification_report(
            class_labels, class_preds, 
            target_names=['Negative', 'Positive'],
            digits=4
        )
        print(report)
        
        # Calculate individual metrics
        acc = accuracy_score(class_labels, class_preds)
        precision = precision_score(class_labels, class_preds, zero_division=0)
        recall = recall_score(class_labels, class_preds, zero_division=0)
        f1 = f1_score(class_labels, class_preds, zero_division=0)
        
        reports.append({
            'Label': label,
            'Accuracy': acc,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        })

    # Create summary table
    reports_df = pd.DataFrame(reports)
    print("\n" + "=" * 60)
    print("📊 SUMMARY TABLE")
    print("=" * 60)
    print(reports_df.to_string(index=False))

    # Save reports to CSV
    reports_df.to_csv(os.path.join(config.OUTPUT_DIR, 'mobilenet_classification_reports.csv'), index=False)
    print("\n✅ Saved: mobilenet_classification_reports.csv")

## 📊 Metrics Comparison Chart

In [ ]:
if len(all_preds) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))

    x = np.arange(len(config.LABELS))
    width = 0.2

    bars1 = ax.bar(x - 1.5*width, reports_df['Accuracy'], width, label='Accuracy', color='#2ecc71')
    bars2 = ax.bar(x - 0.5*width, reports_df['Precision'], width, label='Precision', color='#3498db')
    bars3 = ax.bar(x + 0.5*width, reports_df['Recall'], width, label='Recall', color='#e74c3c')
    bars4 = ax.bar(x + 1.5*width, reports_df['F1-Score'], width, label='F1-Score', color='#9b59b6')

    ax.set_xlabel('Class', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('📊 Binary Classification Metrics - MobileNet V2', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(config.LABELS, fontsize=11)
    ax.legend(fontsize=10)
    ax.set_ylim(0, 1.05)
    ax.grid(True, alpha=0.3, axis='y')

    # Add value labels on bars
    for bars in [bars1, bars2, bars3, bars4]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.2f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    plt.savefig(os.path.join(config.OUTPUT_DIR, 'mobilenet_metrics_comparison.png'), dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Saved: mobilenet_metrics_comparison.png")

## 📈 ROC Curves for All 4 Classes

In [ ]:
if len(all_preds) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    # axes is already 1D for 1x3 subplot

    colors = ['#e74c3c', '#3498db', '#2ecc71']

    for i, label in enumerate(config.LABELS):
        class_probs = all_probs[:, i]
        class_labels = all_labels[:, i]
        
        # Compute ROC curve
        fpr, tpr, _ = roc_curve(class_labels, class_probs)
        roc_auc = auc(fpr, tpr)
        
        # Plot
        axes[i].plot(fpr, tpr, color=colors[i], linewidth=2, 
                     label=f'ROC curve (AUC = {roc_auc:.4f})')
        axes[i].plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
        axes[i].fill_between(fpr, tpr, alpha=0.2, color=colors[i])
        axes[i].set_xlim([0.0, 1.0])
        axes[i].set_ylim([0.0, 1.05])
        axes[i].set_xlabel('False Positive Rate', fontsize=12)
        axes[i].set_ylabel('True Positive Rate', fontsize=12)
        axes[i].set_title(f'🏷️ {label}\nROC Curve', fontsize=14, fontweight='bold')
        axes[i].legend(loc='lower right', fontsize=11)
        axes[i].grid(True, alpha=0.3)

    plt.suptitle(' ROC Curves', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(os.path.join(config.OUTPUT_DIR, 'efficientnet_roc_curves.png'), dpi=300, bbox_inches='tight')
    plt.show()
   

## 🎉 Final Summary

In [ ]:
print("=" * 60)
print("🎉 TRAINING COMPLETE! - MobileNet V2")
print("=" * 60)

if len(all_preds) > 0:
    print(f"""
    🤖 Model: MobileNet V2
    📊 Dataset: CheXpert (4-class: {', '.join(config.LABELS)})
    🏋️ Training samples: {len(train_df)}
    🧪 Validation samples: {len(val_df)}

    📉 Best Validation Loss: {best_val_loss:.4f}
    📈 Final Validation Accuracy: {val_acc:.4f}

    📁 Output Files:
      📊 mobilenet_training_curves.png
      📊 mobilenet_confusion_matrices.png
      📊 mobilenet_metrics_comparison.png
      📊 mobilenet_roc_curves.png
      📄 mobilenet_classification_reports.csv
      🔧 mobilenet_chestxray.pth

    📋 Per-Class Performance:
    """)

    for _, row in reports_df.iterrows():
        print(f"  {row['Label']}: Acc={row['Accuracy']:.4f} | F1={row['F1-Score']:.4f}")

    print("\n✅ All outputs saved successfully!")
else:
    print("⚠️ No results to report.")